[![colab-logo](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/japan-medical-ai/medical-ai-course-materials/blob/master/notebooks/Introduction_to_Neural_Network.ipynb)

# ニューラルネットワークの基礎

ここでは，ニューラルネットワーク (Neural Network) について簡単に紹介していきます．画像認識などに用いられる Convolutional Neural Network (CNN) や，自然言語処理などに用いられる Recurrent Neural Network (RNN) といった手法は，ニューラルネットワークの一種です．

ここではまず，最もシンプルなニューラルネットワークの構造について説明を行ったあと，複数の入力データと望ましい出力の組からなる学習用データセットを準備したとき，どうやってニューラルネットを学習させればよいのか（教師あり学習）について解説を行います．

ニューラルネットワークによって表現される複雑な関数を，現実的な時間で学習する手法についても紹介します．

まずはニューラルネットワークをブラックボックスとして扱ってしまうのではなく，一つ一つ内部で行われる計算を丁寧に調べます．そして，パラメータで特徴づけられた関数で表される線形変換とそれに続く非線形変換を組み合わせて，全体として微分可能な一つの関数を表していることを理解していきます．


## ニューラルネットワークの構造

まずはニューラルネットワークの構造を図式化して見てみましょう．

![01.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/01.png)

この図のひとつひとつの丸い部分のことを**ノード**と呼び，そのノードの縦方向の集まりを**層**と呼びます．そして，一番初めの層を**入力層**，最後の層を**出力層**，そしてその間を**中間層**もしくは**隠れ層**と呼びます．このモデルは入力層，中間層，出力層の３層の構造となっていますが，中間層の数を増やすことでさらに多層のニューラルネットワークを定義することもできます．この例では各層間の全てのノードが結合されているため，**全結合のニューラルネットワーク**とも呼び，ニューラルネットワークの最も基礎的な構造です．

入力変数はこれまでと同様ですが，出力変数の扱い方がこれまでと異なります．例えば，上図では出力層の各ノードがそれぞれ白ワインと赤ワインに対応しています．このようにカテゴリの数だけ出力の変数があるということになります．なぜこのような構造となっているのでしょうか．

![02.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/02.png)

まず，最終層にどのような値が入るのか，具体例を見てみましょう．例えば，年数が3年物でアルコール度数が14度，色合いが0.2，匂いが0.8で表されるワインがあるとします．内部の計算は後述するとして，このようなデータをニューラルネットワークに与えると，白ワイン $y_{1} = 0.15$, 赤ワイン $y_{2}= 0.85$ という値が得られました．このとき，出力値の中で最も大きな値となっている変数に対応するクラス，すなわち「赤ワイン」をこの分類問題におけるこのニューラルネットワークによる予測結果とすることができます．

ここで出力ノードすべての値を合計してみると，1になっていることに気づきます．これは偶然ではなく，そうなるように出力層の値を計算しているためです．つまり，出力層のそれぞれのノードが持つ数値は，入力がそれぞれのクラスに属している確率を表していたのでした．そのため，カテゴリ数と同じ数だけ出力層にはノードが必要となります．

それでは，ここからニューラルネットワークの内部で行われる計算を詳しく見ていきましょう．ニューラルネットワークの各層は，前の層の値に線形変換と非線形変換を順番に施すことで計算されています．まずは，ここで言う線形変換とは何を表すのか，から見ていきましょう．

### 線形変換

![03.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/03.png)

ここで言う線形変換とは，重み行列 ($w$) ×入力ベクトル ($h$) + バイアスベクトル ($b$) のような計算のことを指しています．ここでの掛け算は行列の掛け算であることに注意してください．また，これからは，$h$ が文字としてよく登場しますが，これは隠れ層 (hidden layer) の頭文字である $h$ から来ています．ただし，表記の簡潔にするため入力層（上図における $x_1, x_2, x_3, x_4$）も，$0$層目の隠れ層と考えることにして，以下では $h_{01}, h_{02}, h_{03}, h_{04}$ と表記します．では上図で表される計算を数式で記述してみましょう．

$$
\begin{aligned}
u_{11}&=w_{11}h_{01}+w_{12}h_{02}+w_{13}h_{03}+w_{14}h_{04}+b_{1} \\
u_{12}&=w_{21}h_{01}+w_{22}h_{02}+w_{23}h_{03}+w_{24}h_{04}+b_{2} \\
u_{13}&=w_{31}h_{01}+w_{32}h_{02}+w_{33}h_{03}+w_{34}h_{04}+b_{2}
\end{aligned}
$$

これは，ベクトルと行列の計算として書き直すことができ，

$$
\begin{aligned}
\begin{bmatrix}
u_{11} \\
u_{12} \\
u_{13}
\end{bmatrix}&=\begin{bmatrix}
w_{11} & w_{12} & w_{13} & w_{14} \\
w_{21} & w_{22} & w_{23} & w_{24} \\
w_{31} & w_{32} & w_{33} & w_{34}
\end{bmatrix}\begin{bmatrix}
h_{01} \\
h_{02} \\
h_{03} \\
h_{04}
\end{bmatrix}+\begin{bmatrix}
b_{1} \\
b_{2} \\
b_{3}
\end{bmatrix}\\
{\bf u}_{1}&={\bf W}{\bf h}_{0}+{\bf b}
\end{aligned}
$$

と同じことです．本来は ${\bf W}$ や ${\bf b}$ にもどの層とどの層の間の計算に用いられるものなのかを表す添え字をつけるべき（たとえば${\bf W_{1 \to 2}}$のように）ですが，ここでは簡単のため省略しています．

### 非線形変換

![04.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/04.png)

次に，非線形変換について説明します．線形変換のみでは，上図右のように入力と出力の間が非線形な関係である場合は，両者の間の関係を適切に表現することができません．そこで，ニューラルネットワークでは各層で線形変換に引き続いて非線形変換を施すことで，全体の関数が非線形性を持つようにしています．この非線形変換を行う関数を，ニューラルネットワークの文脈においては **活性化関数** と呼びます．

![05.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/05.png)

ニューラルネットワークでは上図に示す **ロジスティックシグモイド関数**（以下シグモイド関数）
$$
h = f(u) = \dfrac{1}{1+e^{-u}}
$$
が従来，よく用いられてきました．しかし，近年，層数が多いニューラルネットワークではシグモイド関数は活性化関数としてほとんど用いられていません．その理由の一つは，シグモイド関数を活性化関数に採用することで **勾配消失** という現象が起きやすくなり，学習が進行しなくなる問題が発生することがあったためです．これは後で詳述します．これを回避するために，**Rectified Linear Unit (ReLU)** という関数がよく用いられています．これは，以下のような形をした関数です．

![06.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/06.png)
$$
h = f(u) = \max(0, u)
$$
これは，入力が負の値の場合には出力は0で一定であり，正の値の場合は入力をそのまま出力するという関数です．シグモイド関数では，入力が小さな，もしくは大きな値をとった際に，勾配がどんどん小さくなってしまうだろうことがプロットからも明らかに見て取れます．それに対し，ReLU関数は入力の値がいくら大きくなっても，一定の勾配が発生します．これがのちほど紹介する勾配消失という問題に有効に働くのです．

### 数値を見ながら計算の流れを確認

![07.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/07.png)

ここで，上図に書き込まれた具体的な数値を使って，入力 $x_1, x_2, x_3$ から出力 $y$ が計算される過程を確認してみましょう．今は計算を簡略化するためバイアス ${\bf b}$ の計算は省略します．数値例として，${\bf x}^T = \begin{bmatrix} 2 & 3 & 1 \end{bmatrix}$ が与えられた時の出力 $y$ の計算手順を一つ一つ追いかけてみましょう．

重回帰分析では，目的関数のパラメータについての導関数を0とおいて解析的に最適なパラメータを計算できましたが，ニューラルネットワークでは一般的に解析的にパラメータを解くことはできません．その代わり，この導関数の値（勾配）を利用した別の方法でパラメータを逐次的に最適化していきます．

このため，ニューラルネットワークの場合は，まずパラメータを乱数で初期化し，ひとまずデータを入力して目的関数の値を計算します．次にその関数の勾配を計算して，それを利用してパラメータを更新し，その更新後の新しいパラメータを使って再度入力を処理して目的関数の値を計算し…といったことを繰り返し行っていくことになります．今，上の図のグラフの枝に与えられているような数値でパラメータを初期化した状態で，入力層の値に線形変換を施すところまでを考えてみましょう．この計算は，以下のようになります．

$$
\begin{aligned}
u_{11}&=3\times 2+1\times 3+2\times 1=11\\
u_{12}&=-2\times 2-3\times 3-1\times 1=-11
\end{aligned}
$$

次に非線形変換を行う活性化関数としてReLU関数を採用し，以下のように中間層の値を計算します．

$$
\begin{aligned}
h_{11} &= \max(0, 11) = 11 \\
h_{12} &= \max(0, -11)  = 0
\end{aligned}
$$

同様に，出力層の $y$ の値までを計算すると，

$$
y = 3 \times 11 + 2 \times 0 = 33
$$

となります．

さて，次節からは，パラメータを，どうやって更新していくかを見てみましょう．


## 目的関数

ニューラルネットワークでは，微分可能でさえあれば解きたいタスクに合わせて様々な目的関数を利用することができます．

例えば，出力層に$N$個の値を持つニューラルネットワークで回帰問題を解く場合を考えてみましょう．$N$個の出力それぞれ（$y_n (n=1, 2, \dots, N)$）に対して望ましい出力（$t_n (n=1, 2, \dots, N)$）が与えられたとき，目的関数をそれぞれの出力（$y_n$）と対応する正解（$t_n$）の間の **平均二乗誤差（mean squared error）** とすることで，回帰問題を解くことができます．

$$
\mathcal{L} = \dfrac{1}{N} \sum_{n=1}^{N}(t_{n} - y_{n})^{2}
$$

これを最小にするようなパラメータを求めればよいということになります．例えば，上図の例で正解として $t = 20$ が与えられたときの目的関数の値は，

$$
\mathcal{L} = \dfrac{1}{1} (20 - 33)^2 = 169
$$

となります．

一方，分類問題の場合は **交差エントロピー（cross entropy）** が目的関数として多くの場合利用されます．$N$クラスの分類問題を考えるとき，$N$個の出力 $y_n (n=1, 2, \dots, N)$ が入力がそれぞれのクラスに属する確率（$y_i = p(y=i|x)$）を表しているとして，正解が1-hotベクトルで与えられる時，以下の計算で定義されるものがクロスエントロピーです．ただし，${\bf t} = \begin{bmatrix} t_1 & t_2 & \dots & t_N \end{bmatrix}^T$ が1-hotベクトルであるとは，$t_n (n=1, 2, \dots, N)$ のいずれかが1でありそれ以外は0であるようなベクトルのことを言います．このベクトルの1である要素はその要素のインデックスに対応したクラスが正解であることを意味します．例えば，$t_3 = 1$であれば3というインデックスに対応するクラスが正解であることになります．

$$
\mathcal{L} = - \sum_{n=1}^{N}t_{n}\log y_{n}
$$

以下は交差エントロピーの定義について知りたい方だけ参考にしてください．情報理論などで交差エントロピーの定義を知っている方はこの値が交差エントロピーと違うようにみえます．これは以下のように説明できます．$q(y|x)$をニューラルネットワークのモデルが定義する条件付き確率であり，$p(y|x)$を実データの条件付き確率とします．$p(y|x)$は未知ですが，代わりに学習データの経験分布 $p(y|x) = \frac{1}{n} \sum_i I(x =x_i, y=y_i)$であるとします．ただし$I$はディラック関数とよばれ，その等号が成立する時，値が$\infty$，それ以外では$0$であるような関数です．この時，確率分布$p(y|x)$と$q(y|x)$間の交差エントロピーは

$$
\int_{x, y} p(y|x) \log \frac{q(y|x)}{p(y|x)}   dx dy
$$
と定義されます．ここでディラック関数の定義および，$p$だけに依存する項を除くと，先程の交差エントロピーの目的関数が導出されます．


## パラメータの最適化

最適化の方法を考える前に，まず最適化の対象とはなんであったか，再度確認しましょう．ニューラルネットワークにおける“パラメータ”とは，ここまで紹介したシンプルな全結合型ニューラルネットワークの場合，各層の線形変換に用いられていた ${\bf W}$ と ${\bf b}$ のことを指します．

ニューラルネットワークでは，目的関数の各パラメータについての勾配を0とおいて解析的に解くことは一般的には困難です．しかし，実際にデータをニューラルネットワークに入力してその入力の値における目的関数のパラメータについての勾配を数値的に求めることは可能です．この値が分かれば，パラメータをどのように変化させれば，その入力が再び与えられたときに出力される目的関数の値を小さく（または大きく）することができるのか，が分かります．すなわち，この勾配を使ってパラメータの最適化を行うことができます．この方法について説明を行います．

では，まず以下の図を見てください．図中の点線は，パラメータ $w$ を変化させた際の目的関数 $\mathcal{L}$ の値を表しています．この例では簡単のため二次関数の形になっていますが，実際にはもっと複雑な関数であることがほとんどでしょう．この目的関数が最小値を与えるような $w$ は，どのように発見することができるでしょうか．

![13.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/13.png)

前節で説明したように，ニューラルネットワークのパラメータはまず乱数で初期化されます．ここでは，例として $w=3$ という初期化が行われたと考えてみましょう．そうすると，$w=3$における$\mathcal{L}$の勾配 $\frac{\partial \mathcal{L}}{\partial w}$ が求まります．ここでは，仮に $w=3$ における $\frac{\partial \mathcal{L}}{\partial w}$ が $3$ であったとしましょう（このことを$\frac{\partial \mathcal{L}}{\partial w} |_{w=3} = 3$と書きます）．すると，以下の図のように，この $3$ という値は $w=3$ における $\mathcal{L}(w)$ という関数の接線の傾きを表しています．．

![11.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/11.png)

もし，$\mathcal{L}$の値を小さくしたいのであれば，勾配とは逆の方向に $w$ を変化させる，すなわち **$w$ から $\frac{\partial \mathcal{L}}{\partial w}$ を引いてやればよい** でしょう．これがニューラルネットワークのパラメータを目的関数の勾配を用いて更新していく際の基本的な考え方です．このときの $w$ のステップサイズ（更新量）のスケールを調整するために，勾配に **学習率** (learning rate) と呼ばれる値を乗じるのが一般的です．

例えば，今学習率を $0.5$ に設定してみます．そうすると，$w$の更新量は $-$ 学習率 $\times$ 勾配で決まるので，$-0.5 \times 3 = -1.5$ となります．現在 $w=3$ なので，$w \leftarrow w - 1.5$ と更新した後は， $w=1.5$ となります．ここでさらに，この点においても，勾配を求めてみます．その結果が $\frac{\partial \mathcal{L}}{\partial w} |_{w=1.5} = -1$ であったとしましょう．すると，次の更新量は，$- 0.5 \times -(-1) = 0.5$ となります．以上の通りに2回更新したあとは，以下の図のような位置に $w$ はあるでしょう．

![12.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/12.png)

徐々に$\mathcal{L}$が最小値をとるときの$w$の値に近づいていっていることが見て取れます．

このように，「-1 $\times$ 学習率 $\times$ 勾配」を更新量としてパラメータを変化させていくと，パラメータ$w$ は求めたい $\mathcal{L}$ の最小値を与える $w$ に徐々に近づいていきます．このような勾配を用いた目的関数の最小化を行う手法を **勾配降下法** と呼びます．ニューラルネットワークは，基本的に **微分可能な関数のみを層間をつなぐ関数として用いて** 設計されるため，登場する線形変換と非線形変換はすべて微分可能であり，学習データセットを用いて勾配降下法によってパラメータを最適化する方法が適用可能です．

ニューラルネットワークを勾配降下法で最適化する場合は，学習データセット内に存在するすべてのサンプルに対して目的関数の値を計算し，その総和を最小にするようにパラメータを更新する**バッチ最適化**という方法**ではなく**，「**ミニバッチ学習**」と呼ばれる方法をとるのが一般的です．これは，学習データセットからランダムに$k (>0)$個のデータを抽出し，その$k$個のデータに対する目的関数の平均の値を小さくするようパラメータを更新することを，異なる$k$個のデータの組み合わせに対して繰り返し行うことで，データセットに含まれる全てのデータを使った最適化を行うことです．データセット内のデータを1度ずつ，ただし$k$個のかたまりごとに用いて，すべてパラメータ更新に用い終えるまでを一般的に **1エポック** と呼びます．また，このときの$k$をバッチサイズもしくはミニバッチサイズと呼び，このような方法は，**確率的勾配降下法** (SGD: Stocastic Gradient Descent) と呼ばれます．現在ほとんどすべてのニューラルネットワークのための最適化手法はこのSGDをベースとした手法となっています．SGDを用いると，全体の計算時間が劇的に少なくできるだけでなく，下図のように目的関数が凸関数でなかったとしても，“ほとんど確実に”局所最適解に収束することが知られています．

![14.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/14.png)

### パラメータ更新量の算出

![08.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/08.png)

それでは今，上図のような3層の全結合型ニューラルネットワークを考え，1層目と2層目の間の線形変換が ${\bf w}_1, {\bf b}_1$ というパラメータによって表され，2層目と3層目の間の線形変換が ${\bf w}_2, {\bf b}_2$ というパラメータによって表されるとします（図ではバイアス ${\bf b}_1, {\bf b}_2$ は省略されています）．これらをまとめて $\boldsymbol{\Theta}$ と表すことにします．

入力ベクトルは ${\bf x}$，ニューラルネットワークの出力は ${\bf y} \in \mathbb{R}^N$とし，入力 ${\bf x}$ に対応した“望ましい出力”である教師ベクトルを ${\bf t}$ とします．ここで，目的関数には前述の平均二乗誤差関数を用いることとします．さて，パラメータをそれぞれ適当な乱数で初期化したあと，入力 ${\bf x}$ が与えられたときの目的関数の各パラメータについての勾配を計算して，それぞれのパラメータについて更新量を算出してみましょう．

まず，目的関数を改めてベクトル表記を用いて書き下すと，以下のようになります．

$$
\mathcal{L}({\bf y}, {\bf t}) = \frac{1}{N} || {\bf t} - {\bf y} ||_2
$$

ここで，ニューラルネットワーク全体を $f$ と書くことにすると，出力 ${\bf y}$ は

$$
\begin{aligned}
{\bf y} &= f({\bf x}; \boldsymbol{\Theta}) \\
&= a_2 ( {\bf w}_2 a_1({\bf w}_1 {\bf x} + {\bf b}_1) + {\bf b}_2 )
\end{aligned}
$$

と書くことができます．ここで，$a_1, a_2$ はそれぞれ，1層目と2層目の，および2層目と3層目の間で線形変換のあとに施される非線形変換（活性化関数）です．以下，簡単のために，各層間で行われた線形変換の結果を ${\bf u}_1, {\bf u}_2$とし，中間層の値，すなわち ${\bf u}_1$ に活性化関数を適用した結果を ${\bf h}_1$ と書きます．${\bf u}_2$ に活性化関数を適用した結果は ${\bf y}$ です．すると，これらの関係は以下のように整理することができます．

$$
\begin{aligned}
{\bf y} &= a_2({\bf u}_2) \\
{\bf u}_2 &= {\bf w}_2 {\bf h}_1 + {\bf b}_2 \\
{\bf h}_1 &= a_1({\bf u}_1) \\
{\bf u}_1 &= {\bf w}_1 {\bf x} + {\bf b}_1
\end{aligned}
$$

それではまず，出力層に近い方のパラメータ，${\bf w}_2$ についての $\mathcal{L}$ の勾配を求めてみましょう．これは，合成関数の偏微分なので，連鎖率を用いて以下のように展開できます．

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial {\bf w}_2}
&= \frac{\partial \mathcal{L}}{\partial {\bf y}} \frac{\partial {\bf y}}{\partial {\bf w}_2} \\
&= \frac{\partial \mathcal{L}}{\partial {\bf y}} \frac{\partial {\bf y}}{\partial {\bf u}_2} \frac{\partial {\bf u}_2}{\partial {\bf w}_2}
\end{aligned}
$$

この3つの偏微分はそれぞれ，

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial {\bf y}}
&= -\frac{2}{N} ({\bf t} - {\bf y}) \\
\frac{\partial {\bf y}}{\partial {\bf u}_2}
&= \frac{\partial a_2}{\partial {\bf u}_2} \\
\frac{\partial {\bf u}_2}{\partial {\bf w}_2} 
&= {\bf h}_1
\end{aligned}
$$

と求まります．ここで，活性化関数の入力に関する出力の勾配（$\frac{\partial a_2}{\partial {\bf u}_2}$）が登場しました．これは，例えば活性化関数にシグモイド関数を用いる場合は，

$$
a_2({\bf u}_2) = \frac{1}{1 + \exp(-{\bf u}_2)}
$$

を微分すればよく，すなわち

$$
\begin{aligned}
\frac{\partial a_2({\bf u}_2)}{\partial {\bf u}_2}
&= -\frac{-(\exp(-{\bf u}_2))}{(1 + \exp(-{\bf u}_2))^2} \\
&= \frac{1}{1 + \exp(-{\bf u}_2)} \cdot \frac{\exp(-{\bf u}_2)}{1 + \exp(-{\bf u}_2)} \\
&= \frac{1}{1 + \exp(-{\bf u}_2)} \cdot \frac{1 + \exp(-{\bf u}_2) - 1}{1 + \exp(-{\bf u}_2)} \\
&= \frac{1}{1 + \exp(-{\bf u}_2)} (1 - \frac{1}{1 + \exp(-{\bf u}_2)}) \\
&= a_2({\bf u}_2)(1 - a_2({\bf u}_2))
\end{aligned}
$$

となります．シグモイド関数の勾配は，このようにシグモイド関数の出力値を使って簡単に計算することができます．

これで ${\bf w}_2$ の勾配を計算するのに必要な値は全部計算できそうです．では実際にNumPyを使ってこの勾配を計算してみましょう．ここでは簡単のために，バイアスベクトルはすべて0で初期化されているとします．

In [0]:
import numpy as np

# 入力
x = np.array([2, 3, 1])

# 正解
t = np.array([20])

まず，NumPyモジュールを読み込んでから，入力の配列を定義します．ここでは，上図と同じになるように `2, 3, 1` の3つの値を持つ3次元ベクトルを定義しています．また，正解として仮に `20` を与えることにしました．次に，パラメータを定義します．

In [0]:
# 1-2層間のパラメータ
w1 = np.array([[3, 1, 2], [-2, -3, -1]])
b1 = np.array([0, 0])

# 2-3層間のパラメータ
w2 = np.array([[3, 2]])
b2 = np.array([0])

ここでは，以下の4つのパラメータを定義しました．

**1層目と2層目の間の線形変換のパラメータ**

${\bf w}_1 \in \mathbb{R}^{2 \times 3}$ : 3次元ベクトルを2次元ベクトルに変換する行列

${\bf b}_1 \in \mathbb{R}^2$ : 2次元バイアスベクトル

**2層目と3層目の間の線形変換のパラメータ**

${\bf w}_2 \in \mathbb{R}^{1 \times 2}$ : 2次元ベクトルを1次元ベクトルに変換する行列

${\bf b}_2 \in \mathbb{R}^1$ : 1次元バイアスベクトル

それでは，各層の計算を実際に実行してみましょう．

In [0]:
# 中間層の計算
u1 = w1.dot(x) + b1
h1 = 1. / (1 + np.exp(-u1))

# 出力の計算
u2 = w2.dot(h1) + b2
y = 1. / (1 + np.exp(-u2))

print(y)

[0.95257194]


出力は $0.95257194$ と求まりました．つまり，$f([2, 3, 1]^T) = 0.95257194$ ということになります．次に，上で求めた

$$
\frac{\partial \mathcal{L}}{\partial {\bf w}_2}
= \frac{\partial \mathcal{L}}{\partial {\bf y}} \frac{\partial {\bf y}}{\partial {\bf u}_2} \frac{\partial {\bf u}_2}{\partial {\bf w}_2}
$$

の右辺の3つの偏微分をそれぞれ計算してみましょう．

In [0]:
# dL / dy
g_Ly = -2 / 1 * (t - y)

# dy / du_2
g_yu2 = y * (1 - y)

# du_2 / dw_2
g_u2w2 = h1

これらを掛け合わせれば，求めたかったパラメータ ${\bf w}_2$ についての勾配を得ることができます．

In [0]:
# dL / dw_2: 求めたい勾配
g_Lw2 = g_Ly * g_yu2 * g_u2w2

print(g_Lw_2)

[-1.72104507e+00 -1.43112111e-06]


勾配が求まりました．これが $\frac{\partial \mathcal{L}}{\partial {\bf w}_2}$ の値です．これを学習率でスケールさせたものを使えば，パラメータ ${\bf w}_2$ を更新することができます．更新式は，具体的には以下のようになります．

$$
{\bf w}_2 \leftarrow {\bf w}_2 - \eta \frac{\partial \mathcal{L}}{\partial {\bf w}_2}
$$

$\eta$ が学習率と呼ばれるもので，これが大きすぎると，繰り返しパラメータ更新を行っていく中で目的関数の値が振動したり，発散したりしてしまいます．小さすぎると，収束に時間がかかってしまいます．そのため，この学習率を適切に決定することがニューラルネットワークの学習においては非常に重要となります．

次に，${\bf w}_1$ についての勾配も求めてみましょう．ここで，これは，以下のように計算できるはずです．

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial {\bf w}_1}
&= \frac{\partial \mathcal{L}}{\partial {\bf y}} \frac{\partial {\bf y}}{\partial {\bf w}_1} \\
&=
\frac{\partial \mathcal{L}}{\partial {\bf y}}
\frac{\partial {\bf y}}{\partial {\bf u}_2}
\frac{\partial {\bf u}_2}{\partial {\bf w}_1} \\
&=
\frac{\partial \mathcal{L}}{\partial {\bf y}}
\frac{\partial {\bf y}}{\partial {\bf u}_2}
\frac{\partial {\bf u}_2}{\partial {\bf h}_1}
\frac{\partial {\bf h}_1}{\partial {\bf w}_1} \\
&=
\frac{\partial \mathcal{L}}{\partial {\bf y}}
\frac{\partial {\bf y}}{\partial {\bf u}_2}
\frac{\partial {\bf u}_2}{\partial {\bf h}_1}
\frac{\partial {\bf h}_1}{\partial {\bf u}_1}
\frac{\partial {\bf u}_1}{\partial {\bf w}_1}
\end{aligned}
$$

この5つの偏微分のうち初めの1つはすでに求めました．残りの3つは，それぞれ，

$$
\begin{aligned}
\frac{\partial {\bf y}}{\partial {\bf u}_2}
&= {\bf y}(1 - {\bf y}) \\
\frac{\partial {\bf u}_2}{\partial {\bf h}_1}
&= {\bf w}_2 \\
\frac{\partial {\bf h}_1}{\partial {\bf u}_1}
&= {\bf h}_1(1 - {\bf h}_1) \\
\frac{\partial {\bf u}_1}{\partial {\bf w}_1}
&= {\bf x}
\end{aligned}
$$

と計算することができます．では，さっそく実際にNumPyを用いて計算を実行してみましょう．

In [0]:
g_yu2 = y * (1 - y)
g_u2h1 = w2
g_h1u1 = h1 * (1 - h1)
g_u1w1 = x

# 上から du1 / dw1 の直前までを一旦計算
g_Lu1 = g_Ly * g_yu_2 * g_u_2h_1 * g_h_1u_1

# g_u1w1は (3,) というshapeなので，g_u1w1[None]として(1, 3)に変形
g_u1w1 = g_u1w1[None]

# dL / dw_1: 求めたい勾配
g_Lw1 = g_Lu1.T.dot(g_u1w1)

print(g_Lw1)

[[-1.72463398e-04 -2.58695098e-04 -8.62316992e-05]
 [-5.72447970e-06 -8.58671954e-06 -2.86223985e-06]]


計算ができました．これが $\frac{\partial \mathcal{L}}{\partial {\bf w}_1}$ の値です．これを用いて，${\bf w}_2$ と同様に以下のような更新式でパラメータ ${\bf w}_1$ の更新をすることができます．

$$
{\bf w}_1 \leftarrow {\bf w}_1 - \eta \frac{\partial \mathcal{L}}{\partial {\bf w}_1}
$$

## 誤差逆伝播法（バックプロパゲーション）

ここまでで，勾配を手計算により導出して実際に数値計算を行うということを体験しました．しかし，これは層数が増えてくれば増えてくるほど，一つ一つのパラメータに関する勾配を手計算で求めることは大変になっていきます．

しかし，ここまでで何度か行ってきた合成関数の偏微分を連鎖率によって複数の偏微分の積の形に変形する行程は自動的に計算できそうです．

そこで，ここまでの説明で用いていた3層全結合型ニューラルネットワークをもう一度見直して，更新量がどのように計算されていたか動画で確かめてみましょう．

![backpropagation](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/backpropagation.gif)

ここでは，目的関数の出力を $l = \mathcal{L}({\bf y}, {\bf t})$ としています．この図の丸いノードは変数を表し，四角いノードは関数を表しています．上で説明したように，このニューラルネットワーク全体を $f$ と表す場合，各層間の線形変換を $f_1$, $f_2$ と表すと，$f$ は以下のような関数合成で書くことができることになります．

$$
f = a_2 \circ f_2 \circ a_1 \circ f_1
$$

ここで，$\circ$という記号は関数合成を意味します．例えば，関数$f$を$x$に適用した結果にさらに関数$g$を適用するという操作は，

$$
g(f(x))
$$

と記述できますが，適用する関数がさらに多くなると括弧が大量に登場して記述が煩雑になってしまいます．そのため，このような連続した関数の適用の操作を関数の合成記号$\circ$を用いて

$$
(g \circ f)(x)
$$

と書くことがあります．これにより多くの関数を適用する際には適用の順番や関数の個数などが見やすくなります．

さて，ニューラルネットワークの場合，**この全体（$f$）を構成する一つ一つの関数（$a_2, f_2, a_1, f_1$）のそれぞれが全て，それぞれの入力について微分可能になっているはずです**．そのため，事前にそれらの導関数を計算しておけば，新しい入力が渡されたとき，それを導関数に代入するだけですぐに入力に関する勾配の値を計算することができます．例えば，図中の$\frac{\partial \mathcal{L}}{\partial {\bf y}}$ や，$\frac{\partial a_2}{\partial {\bf u}_2}$ などは，$\mathcal{L}$ や $a_2$ に入力が与えられた時点で計算することができます．

また，このニューラルネットワークに入力データ ${\bf x}$ を与えて，出力へ向けて順番に計算を行っていくことでデータが伝播していくこと（**上の図で青色の矢印で表された左から右方向への計算**）を，**順伝播（forward computation）** といいます．

順伝播が進行していくと，すでに通過した関数に対してはその入力に関する勾配の値が求まっていきます．そして，順伝播計算の終着点にあるのは目的関数の計算です．このとき，計算グラフの途中にでてきた2個めの線形変換 $f_2$ に着目すると，この関数が持つパラメータ ${\bf w}_2$ についての目的関数 $\mathcal{L}$ の勾配は，**この関数より先にあるすべての関数の勾配をかけ合わせたもの**＝$\frac{\partial a_2}{\partial {\bf u}_2} \frac{\partial \mathcal{L}}{\partial {\bf y}}$ に，**この関数に対するパラメータについての勾配**＝$\frac{\partial f_2}{\partial {\bf w}_2}$ を掛け合わせれば，求められるということが分かります．上の図中にも式で示されている，以下の計算のことです．

$$
\frac{\partial \mathcal{L}}{\partial {\bf w}_2}
=
\frac{\partial f_2}{\partial {\bf w}_2}
\frac{\partial a_2}{\partial {\bf u}_2}
\frac{\partial \mathcal{L}}{\partial {\bf y}}
$$

つまり，**パラメータ ${\bf w}_2$ の更新量＝目的関数に対する ${\bf w}_2$ についての勾配** は，この図が示すように，**出力側から順に，逆向きに各関数の入力に関する勾配を計算していき，それを掛け合わせていく**ことで計算できるわけです．

このように，微分の連鎖率の仕組みを用いてニューラルネットワーク $f$ の微分を計算し，パラメータの更新量を求めることを実際のデータを用いて繰り返し行ってパラメータ最適化を行うアルゴリズムを，**誤差逆伝播法（backpropagation）** と呼びます．

また，こうしてみるとニューラルネットワークとは「**微分可能な関数によって構成される計算グラフ**」であるということがよく分かります．


## 勾配消失

活性化関数について初めに触れた際，シグモイド関数には勾配消失という現象が起きやすくなるという問題があり，現在はあまり使われていないと説明をしました．その理由についてもう少し詳しく見ていきましょう．

上で既に計算した，シグモイド関数の導関数を思い出してみます．

$$
\begin{aligned}
f\left( u\right) &=\dfrac {1}{1+e^{-u}} \\
f'\left( u\right) &= f\left( u\right) \left( 1-f\left( u\right) \right)
\end{aligned}
$$

さて，この導関数を入力変数に関してプロットしてみると，下記のようになります．

![09.png](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/09.png)

この図を見ると，**シグモイド関数の勾配の最大値は0.25** であることに気づきます．それに対し，前述のReLU関数の場合，入力変数が0より大きければ常に勾配は1となるので，その最大値は1でした．

これにどのような差があるかを考えてみます．各パラメータの更新量を求めるには，そのパラメータが使われた関数よりも**先のすべての関数の勾配をかけ合わせたもの** に，さらに自らの勾配を掛け合わせる必要がありました．しかし，活性化関数にシグモイド関数を用いた場合，線形変換が計算グラフ中に現れるたびにそれに続いて現れる活性化関数の勾配が，どんなに大きくても0.25であるため，活性化関数が登場するたびに目的関数の勾配は少なくとも0.25倍ずつ小さくなっていくわけです．これは，層数が増えていけばいくほど最初の方のパラメータについての勾配は$0$に近づくという問題があります．別の言い方をすると，下の層のパラメータをどのように動かしても出力の値がほとんど変わらなくなることを意味します．

今回は3層のニューラルネットワークを用いて説明を行っていましたが，もし4層の場合，一番入力に近い線形変換のパラメータの勾配を求めようとすると，少なくとも目的関数の勾配が $0.25 \times 0.25 = 0.0625$ 倍されることとなります．結果として，ディープラーニングと呼ばれる分野でしばしば用いられているような，さらに多くの層を積み重ねたニューラルネットワークを訓練したい場合には，活性化関数としてシグモイド関数を使用すると，**目的関数の勾配が入力に近いパラメータへほとんど伝わらなくなっていき**，勾配降下法による最適化で用いるパラメータの更新量の値が極端に小さくなって，どんなに目的関数が大きなエラーを報告していようとも，ほとんどパラメータの値が更新されないということが起きます．これを**勾配消失**と呼び，長らく深い（十数層を超える）ニューラルネットワークの学習が困難であった一つの要因でした．